In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
url = 'https://hummingbirdscanada.ca/provinces/britishcolumbia/plants'

r = requests.get(url)
content = r.content


In [3]:
soup = BeautifulSoup(content, 'html.parser')

In [4]:
biennials = soup.select("#Biennials td")
perennials = soup.select("#Perennials td")
shrubs = soup.select("#Shrubs td")
trees = soup.select("#Trees td")
vines = soup.select("#vines td")

In [5]:
all_types = {"biennials": biennials,"perennials": perennials, "shrubs": shrubs,"trees": trees, "vines": vines}
allPlants = pd.DataFrame(columns = ("type", "latin_name", 'common_name', 'colour', 'bloom_period'))
def html_to_df(plants):
    global allPlants
    
    for plant in plants:
        count_plant = int(len(plants[plant])/4)
        plant_data = plants[plant]

        for i in range(count_plant):
            index = 4*i
            row = pd.DataFrame({"type": [plant], "latin_name": [plant_data[index].text.strip()], "common_name": [plant_data[index+1].text.strip()], 
                                "colour": [plant_data[index+2].text.strip()], "bloom_period": [plant_data[index+3].text.strip()]})
            allPlants = pd.concat([allPlants, row], ignore_index=True)  


In [6]:
html_to_df(all_types)

allPlants

,type,latin_name,common_name,colour,bloom_period
0,biennials,Ipomopsis aggregata,Scarlet Gila,Red,June-August
1,perennials,Allium cernuum,Nodding onion,Purple,May-June
2,perennials,Aquilegia formosa,Western Red Columbine,Red/Orange/Yellow,May-June
3,perennials,Arctostaphylos uva-ursi,Kinnikinnick,Pink/white,July-August
4,perennials,Astragalus canadensis,Canada milk vetch,White,July-August
5,perennials,Castilleja elmeri,Elmer's paintbrush,Red,July-August
6,perennials,Castilleja gracillima,Slender paintbrush,Red and yellow,July-August
7,perennials,Castilleja hispida,Harsh Indian paintbrush,Orange/red,July-August
8,perennials,Castilleja miniata,Common red paintbrush,Orange/red,July-August
9,perennials,Castilleja raupii,Raup's paintbrush,Mauve,June-July


In [7]:
allPlants.drop(labels=[6], axis=0, inplace=True)

## Attempt for checking

Latin_Names = allPlants["latin_name"]
type(Latin_Names)

Latin_Names = Latin_Names.tolist()

Latin_Names2 = [LN_X.replace(" ", "_") for LN_X in Latin_Names]

Latin_Names3 = Latin_Names2[0:5]

def wbsExists(url):
    try:
        response = requests.head(url, timeout=1)  # Sends a HEAD request
        return response.status_code == 200
    except requests.RequestException:
        return False

def check_wikipedia_page_exists(url):
    try:
        response = requests.get(url, timeout=5, allow_redirects=True)
        
        if response.status_code == 200 and "wikipedia.org/wiki/" in response.url:
            return True
        else:
            return False
    except requests.RequestException:
        return False




wbsExists("https://en.wikipedia.org/wiki/Castilleja_raupii")

Latin_Names_Exist = []
urls = []

for lnx in Latin_Names2:
    url = "https://en.wikipedia.org/wiki/" + lnx
    urls.append(url)

for url in urls:
    ext = check_wikipedia_page_exists("https://en.wikipedia.org/wiki/Ipomopsis_aggregata")
    Latin_Names_Exist.append(ext)

print(sum(Latin_Names_Exist))

urls

## Continue

In [12]:
attribution = ([
    "Public Domain: Arches National Park",
    "Fritz Flohr Reynolds",
    "Walter Siegmund",
    "Jörg Hempel",
    "Matt Lavin",
    
    "Walter Siegmund",
    "Walter Siegmund",
    "Jane S. Richardson",
    "Malcom Manners",

    "J Schmidt",
    "J Brew",
    "Danny Steven S.",
    "Wikipedia contributor: kallerna",
    "Walter Siegmund",

    "Wikipedia contributor: Flowersinmyyard",
    "Mason Brock",
    "Walter Siegmund",
    "Debbie: Flickr user shesnuckinfuts",
    "Krzysztof Golik",

    "Walter Siegmund",
    "Walter Siegmund",
    "Matt Lavin",
    "Thayne Tuason",
    "Walter Siegmund",

    "Shane Johnson",
    "USDA NRCS",
    "S. Rae",
    "Jack Bindernagel",
    "Kurt Stüber",
    
    "Matt Berger",
    "Patrick Alexander",
    "John A Haskins",
    "Walter Siegmund",
    "Thayne Tuason",

    "Krzysztof Ziarnek",
    "Michael Wolf",
    "Thayne Tuason",
    "Doug Murphy",
    "Gordon Leppig & Andrea J. Pickart",

    "Krzysztof Ziarnek",
    "Joaquim Alves Gaspar",
    "Colin Durfee",
    "Krzysztof Ziarnek",
    "Walter Siegmund",

    "J Brew",
    "Robert Flogaus-Faust",
    "Matt Lavin",
    "Franz Xaver",
    "Walter Siegmund",

    "Friends of Henrietta Park"
    
])

allPlants["attribution"] = attribution

In [13]:
photo = ([
    "https://upload.wikimedia.org/wikipedia/commons/thumb/8/85/Scarlet_Gilia_%287164605030%29.jpg/640px-Scarlet_Gilia_%287164605030%29.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/Allium_cernuum_-_Nodding_Onion_3.jpg/640px-Allium_cernuum_-_Nodding_Onion_3.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/e/e0/Aquilegia_formosa_14962.JPG/640px-Aquilegia_formosa_14962.JPG",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Arctostaphylos_uva-ursi_LC0146.jpg/640px-Arctostaphylos_uva-ursi_LC0146.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/5/52/Astragalus_canadensis_%285144332268%29.jpg/640px-Astragalus_canadensis_%285144332268%29.jpg",
    
    "https://upload.wikimedia.org/wikipedia/commons/thumb/6/63/Castilleja_elmeri_7878.JPG/640px-Castilleja_elmeri_7878.JPG",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/4/43/Castilleja_hispida_39019.JPG/640px-Castilleja_hispida_39019.JPG",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/e/e1/Castilleja_miniata_close.jpg/640px-Castilleja_miniata_close.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/c/ca/Castilleja_raupii.jpg/640px-Castilleja_raupii.jpg",

    "https://upload.wikimedia.org/wikipedia/commons/thumb/5/58/Castillejarhexifolia.jpg/640px-Castillejarhexifolia.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/5/50/Castilleja_rupicola_1.jpg/640px-Castilleja_rupicola_1.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a1/Dicentra_formosa_by_Danny_S._-_001.JPG/640px-Dicentra_formosa_by_Danny_S._-_001.JPG",
    "https://en.wikipedia.org/wiki/Chamaenerion_angustifolium#/media/File:Maitohorsma_(Epilobium_angustifolium).JPG",
    "https://en.wikipedia.org/wiki/Erythranthe_alsinoides#/media/File:Mimulus_alsinoides_2411.JPG",
    
    "https://en.wikipedia.org/wiki/Erythronium_oregonum#/media/File:Erythronium_oregonum.jpg",
    "https://en.wikipedia.org/wiki/Lilium_philadelphicum#/media/File:Lilium_philadelphicum_var._philadelphicum.jpg",
    "https://en.wikipedia.org/wiki/Linnaea#/media/File:Linnaea_borealis_1190.JPG",
    "https://en.wikipedia.org/wiki/Lupinus_arcticus#/media/File:Lupinus_arcticus.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/b/b1/Lupinus_polyphyllus_in_Canterbury_Region_07.jpg/640px-Lupinus_polyphyllus_in_Canterbury_Region_07.jpg",
    
    "https://en.wikipedia.org/wiki/Erythranthe_lewisii#/media/File:Mimulus_lewisii_8189.JPG",
    "https://en.wikipedia.org/wiki/Nothochelone#/media/File:Nothochelone_nemorosa_0945.JPG",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dc/Penstemon_albertinus_%2828467312652%29.jpg/640px-Penstemon_albertinus_%2828467312652%29.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Penstemon_confertus_near_Squilchuck_State_Park_Chelan_County_Washington_1.jpg/640px-Penstemon_confertus_near_Squilchuck_State_Park_Chelan_County_Washington_1.jpg",
    "https://en.wikipedia.org/wiki/Penstemon_davidsonii#/media/File:Penstemon_davidsonii_6271.JPG",
    
    "https://upload.wikimedia.org/wikipedia/commons/thumb/d/db/Penstemon_ellipticus_J01.jpg/640px-Penstemon_ellipticus_J01.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/2/2f/Plants_OB_924_%2838165836164%29.jpg/640px-Plants_OB_924_%2838165836164%29.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/b/ba/Penstemon_fructicosus_var._serratus_-_Flickr_-_S._Rae.jpg/640px-Penstemon_fructicosus_var._serratus_-_Flickr_-_S._Rae.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/3/31/Penstemon_gormanii_B01.jpg/640px-Penstemon_gormanii_B01.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c2/Penstemon_gracilis4.jpg/640px-Penstemon_gracilis4.jpg",

    "https://upload.wikimedia.org/wikipedia/commons/thumb/2/2b/Penstemon_lyallii_-_Matt_Berger_01.jpg/640px-Penstemon_lyallii_-_Matt_Berger_01.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/5/5e/Penstemon_nitidus_-_Patrick_Alexander_01.jpg/640px-Penstemon_nitidus_-_Patrick_Alexander_01.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/c/ce/Penstemon_ovatus_-_John_A_Haskins_01.jpg/640px-Penstemon_ovatus_-_John_A_Haskins_01.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/e/e4/Penstemon_procerus_21933.JPG/640px-Penstemon_procerus_21933.JPG",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/7/72/Penstemon_pruinosus_1.jpg/640px-Penstemon_pruinosus_1.jpg",

    "https://upload.wikimedia.org/wikipedia/commons/thumb/d/d0/Penstemon_richardsonii_kz01.jpg/640px-Penstemon_richardsonii_kz01.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Penstemon_serrulatus_IMG_1954.jpg/640px-Penstemon_serrulatus_IMG_1954.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/2/24/Stachys_cooleyae_1.jpg/640px-Stachys_cooleyae_1.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/7/70/Holodiscus_discolor_p1216.jpg/640px-Holodiscus_discolor_p1216.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/1/12/Lonicera_involucrata_FWS-01.jpg/640px-Lonicera_involucrata_FWS-01.jpg",

    "https://upload.wikimedia.org/wikipedia/commons/thumb/f/f3/Oemleria_cerasiformis_kz08.jpg/640px-Oemleria_cerasiformis_kz08.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/d/d1/Philadelphus_July_2013-1.jpg/640px-Philadelphus_July_2013-1.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/e/e2/Ribes_lobbii_-_Flickr_-_Colin_Durfee.jpg/640px-Ribes_lobbii_-_Flickr_-_Colin_Durfee.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a5/Ribes_sanguineum_%27Pulborough_Scarlet_Variegata%27_kz03.jpg/640px-Ribes_sanguineum_%27Pulborough_Scarlet_Variegata%27_kz03.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/7/72/Rosa_nutkana_07513.JPG/640px-Rosa_nutkana_07513.JPG",
    
    "https://upload.wikimedia.org/wikipedia/commons/thumb/a/ad/Thimbleberry_%28Rubus_parviflorus%29_-_Flickr_-_brewbooks.jpg/640px-Thimbleberry_%28Rubus_parviflorus%29_-_Flickr_-_brewbooks.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a1/Rubus_spectabilis_4_RF.jpg/640px-Rubus_spectabilis_4_RF.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/8/82/Symphoricarpos_albus_%285062664930%29.jpg/640px-Symphoricarpos_albus_%285062664930%29.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/0/06/Vaccinium_ovatum_2.jpg/640px-Vaccinium_ovatum_2.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/8/8b/Acer_circinatum_6925.JPG/640px-Acer_circinatum_6925.JPG",

    "https://henriettapark.org/wp-content/uploads/2021/07/IMG_0473.jpg"
    
])


allPlants["photo"] = photo

In [14]:
allPlants

,type,latin_name,common_name,colour,bloom_period,attribution,photo
0,biennials,Ipomopsis aggregata,Scarlet Gila,Red,June-August,Public Domain: Arches National Park,https://upload.wikimedia.org/wikipedia/commons...
1,perennials,Allium cernuum,Nodding onion,Purple,May-June,Fritz Flohr Reynolds,https://upload.wikimedia.org/wikipedia/commons...
2,perennials,Aquilegia formosa,Western Red Columbine,Red/Orange/Yellow,May-June,Walter Siegmund,https://upload.wikimedia.org/wikipedia/commons...
3,perennials,Arctostaphylos uva-ursi,Kinnikinnick,Pink/white,July-August,Jörg Hempel,https://upload.wikimedia.org/wikipedia/commons...
4,perennials,Astragalus canadensis,Canada milk vetch,White,July-August,Matt Lavin,https://upload.wikimedia.org/wikipedia/commons...
5,perennials,Castilleja elmeri,Elmer's paintbrush,Red,July-August,Walter Siegmund,https://upload.wikimedia.org/wikipedia/commons...
7,perennials,Castilleja hispida,Harsh Indian paintbrush,Orange/red,July-August,Walter Siegmund,https://upload.wikimedia.org/wikipedia/commons...
8,perennials,Castilleja miniata,Common red paintbrush,Orange/red,July-August,Jane S. Richardson,https://upload.wikimedia.org/wikipedia/commons...
9,perennials,Castilleja raupii,Raup's paintbrush,Mauve,June-July,Malcom Manners,https://upload.wikimedia.org/wikipedia/commons...
10,perennials,Castilleja rhexiifolia,Alpine paintbrush,Purple,June-July,J Schmidt,https://upload.wikimedia.org/wikipedia/commons...
